In [1]:
%matplotlib inline

In [2]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

2023-01-31 11:48:35.014380: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-31 11:48:43.046338: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-31 11:48:43.690417: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [3]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os
from scipy.sparse import csr_matrix

import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

2023-01-31 11:49:35.036099: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-31 11:49:35.036268: W external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)


['default', 'despine']


In [4]:
output_dir = "/lustre/groups/ml01/workspace/moscot_paper/pancreas/driver_features"

In [5]:
tp0 = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas/JASPAR_TemporalProblem.pkl")

In [6]:
adata_shared_embedding = sc.read_h5ad("/lustre/groups/ml01/workspace/moscot_paper/pancreas/embeddings/pancreas_shared_embeddings.h5ad")

In [7]:
adata_imputed_counts = sc.read_h5ad("/lustre/groups/ml01/workspace/moscot_paper/pancreas/embeddings/pancreas_ATAC_imputation.h5ad")

In [8]:
adata_imputed_counts = adata_imputed_counts[adata_imputed_counts.obs_names.isin(tp0.adata.obs_names)]

In [9]:
adata_imputed_counts.layers["log_imputed"] = sc.pp.log1p(adata_imputed_counts.layers["X_normalized"])

In [10]:
mask = tp0.adata[:, tp0.adata.var["modality"]=="ATAC"].var_names.isin(adata_imputed_counts.var_names)

In [11]:
layer = np.zeros(tp0.adata[:,tp0.adata.var["modality"]=="ATAC"].shape)
layer[:, mask] = adata_imputed_counts.layers["log_imputed"]

In [12]:
adata_gex = tp0.adata[:, tp0.adata.var["modality"]=="GEX"]

In [13]:
sc.pp.normalize_total(adata_gex)
sc.pp.log1p(adata_gex)

/home/icb/dominik.klein/miniconda3/envs/jax_gpu3/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [14]:
tp0.adata.layers["ATAC_imputed"] = csr_matrix(np.hstack((adata_gex.X.A, layer)))

In [15]:
peaks = list(tp0.adata[:,tp0.adata.var["modality"]=="ATAC"].var_names)

In [16]:
genes = list(tp0.adata[:,tp0.adata.var["modality"]=="GEX"].var_names)

# Delta

In [17]:
tp0.pull(14.5, 15.5, data="celltype", subset="Delta", key_added = "Delta_pull")

In [18]:
delta_drivers = tp0.compute_feature_correlation(obs_key="Delta_pull", layer="ATAC_imputed", features=genes)

In [19]:
delta_drivers

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
Sst,0.560592,0.000000e+00,0.000000e+00,0.546544,0.574325
Hhex,0.427705,0.000000e+00,0.000000e+00,0.411011,0.444111
Spock3,0.297227,3.033380e-193,1.482615e-189,0.278651,0.315581
Mef2c,0.288142,4.758520e-181,1.744354e-177,0.269461,0.306606
Masp1,0.273336,3.595275e-162,1.054350e-158,0.254491,0.291973
...,...,...,...,...,...
Sox4,-0.093665,9.958275e-20,1.390649e-17,-0.113703,-0.073551
Tmsb4x,-0.102620,2.183421e-23,3.811369e-21,-0.122619,-0.082538
Neurog3,-0.110658,5.863641e-27,1.283262e-24,-0.130619,-0.090607
Setbp1,-0.112883,5.383484e-28,1.273194e-25,-0.132833,-0.092842


In [20]:
delta_drivers = tp0.compute_feature_correlation(obs_key="Delta_pull", layer="ATAC_imputed", features=genes)

In [21]:
delta_drivers

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
Sst,0.560592,0.000000e+00,0.000000e+00,0.546544,0.574325
Hhex,0.427705,0.000000e+00,0.000000e+00,0.411011,0.444111
Spock3,0.297227,3.033380e-193,1.482615e-189,0.278651,0.315581
Mef2c,0.288142,4.758520e-181,1.744354e-177,0.269461,0.306606
Masp1,0.273336,3.595275e-162,1.054350e-158,0.254491,0.291973
...,...,...,...,...,...
Sox4,-0.093665,9.958275e-20,1.390649e-17,-0.113703,-0.073551
Tmsb4x,-0.102620,2.183421e-23,3.811369e-21,-0.122619,-0.082538
Neurog3,-0.110658,5.863641e-27,1.283262e-24,-0.130619,-0.090607
Setbp1,-0.112883,5.383484e-28,1.273194e-25,-0.132833,-0.092842


In [22]:
delta_drivers.to_csv(os.path.join(output_dir, "driver_delta_genes.csv"))

In [23]:
delta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Delta_pull", layer="ATAC_imputed", features="mouse")

In [24]:
delta_drivers_tfs

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
Hhex,0.427705,0.000000e+00,0.000000e+00,0.411011,0.444111
Mef2c,0.288142,4.758520e-181,3.159657e-178,0.269461,0.306606
Arg1,0.246169,1.261756e-130,5.585371e-128,0.227047,0.265101
Isl1,0.224078,8.464530e-108,2.810224e-105,0.204754,0.243228
Meis2,0.136542,2.491647e-40,5.514846e-38,0.116611,0.156363
...,...,...,...,...,...
Nkx2-2,-0.093519,1.134729e-19,1.255767e-17,-0.113558,-0.073405
Sox4,-0.093665,9.958275e-20,1.202235e-17,-0.113703,-0.073551
Neurog3,-0.110658,5.863641e-27,9.733644e-25,-0.130619,-0.090607
Setbp1,-0.112883,5.383484e-28,1.021324e-25,-0.132833,-0.092842


In [25]:
delta_drivers_tfs.to_csv(os.path.join(output_dir, "driver_delta_tfs.csv"))

In [26]:
delta_drivers_peaks = tp0.compute_feature_correlation(obs_key="Delta_pull", layer="ATAC_imputed", features=peaks)

In [27]:
delta_drivers_peaks

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
16-72640851-72641699,0.169180,2.295021e-61,2.887504e-56,0.149438,0.188787
3-39062154-39063049,0.162776,7.295773e-57,4.589625e-52,0.142994,0.182428
19-4124163-4124803,0.159790,7.930528e-55,3.325958e-50,0.139990,0.179463
17-81018425-81019354,0.158353,7.332750e-54,2.306443e-49,0.138543,0.178035
10-5865738-5866637,0.153845,6.828069e-51,1.718161e-46,0.134009,0.173558
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [28]:
delta_drivers_peaks.to_csv(os.path.join(output_dir, "driver_delta_peaks.csv"))

# Epsilon

In [29]:
tp0.pull(14.5, 15.5, data="celltype", subset="Epsilon", key_added="Epsilon_pull")

In [30]:
eps_drivers = tp0.compute_feature_correlation(obs_key="Epsilon_pull", layer="ATAC_imputed", features=genes)

In [31]:
eps_drivers

,Epsilon_pull_corr,Epsilon_pull_pval,Epsilon_pull_qval,Epsilon_pull_ci_low,Epsilon_pull_ci_high
Mboat4,0.572580,0.000000e+00,0.000000e+00,0.558806,0.586037
Lrrtm3,0.538616,0.000000e+00,0.000000e+00,0.524080,0.552839
Irs4,0.524046,0.000000e+00,0.000000e+00,0.509196,0.538583
Ghrl,0.461645,0.000000e+00,0.000000e+00,0.445558,0.477435
Ctnna3,0.441094,0.000000e+00,0.000000e+00,0.424634,0.457263
...,...,...,...,...,...
Igf1r,-0.132164,7.257493e-38,1.033171e-35,-0.152011,-0.112211
Npepl1,-0.135276,1.310817e-39,1.922051e-37,-0.155105,-0.115339
Stx16,-0.137034,1.301270e-40,1.967063e-38,-0.156852,-0.117105
Glis3,-0.138536,1.763731e-41,2.722273e-39,-0.158345,-0.118615


In [32]:
eps_drivers.to_csv(os.path.join(output_dir, "driver_eps_genes.csv"))

In [33]:
eps_drivers_tfs = tp0.compute_feature_correlation(obs_key="Epsilon_pull", layer="ATAC_imputed", features="mouse")

In [34]:
eps_drivers_tfs

,Epsilon_pull_corr,Epsilon_pull_pval,Epsilon_pull_qval,Epsilon_pull_ci_low,Epsilon_pull_ci_high
Arg1,0.414454,0.000000e+00,0.000000e+00,0.397537,0.431089
Isl1,0.350129,4.351758e-274,2.889567e-271,0.332231,0.367774
Nuak2,0.266156,1.886981e-153,8.353038e-151,0.247235,0.284874
Arx,0.248868,1.382448e-133,4.589726e-131,0.229773,0.267772
Mctp2,0.239500,1.791893e-123,4.759269e-121,0.220315,0.258500
...,...,...,...,...,...
Sox5,-0.092460,2.915587e-19,1.683435e-17,-0.112503,-0.072342
Pax4,-0.102755,1.912022e-23,1.410647e-21,-0.122753,-0.082673
Prdm16,-0.103624,8.088641e-24,6.318656e-22,-0.123619,-0.083546
Glis3,-0.138536,1.763731e-41,1.801720e-39,-0.158345,-0.118615


In [35]:
eps_drivers_tfs.to_csv(os.path.join(output_dir, "driver_eps_tfs.csv"))

In [36]:
eps_drivers_peaks = tp0.compute_feature_correlation(obs_key="Epsilon_pull", layer="ATAC_imputed", features=peaks)

In [37]:
eps_drivers_peaks

,Epsilon_pull_corr,Epsilon_pull_pval,Epsilon_pull_qval,Epsilon_pull_ci_low,Epsilon_pull_ci_high
10-64082077-64082994,0.504819,0.0,0.0,0.489571,0.519758
2-155892590-155893351,0.503065,0.0,0.0,0.487781,0.518040
4-57291831-57292786,0.494789,0.0,0.0,0.479339,0.509932
3-79085915-79086814,0.494417,0.0,0.0,0.478960,0.509568
9-11562792-11563740,0.489033,0.0,0.0,0.473469,0.504292
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [38]:
eps_drivers_peaks.to_csv(os.path.join(output_dir, "driver_eps_peaks.csv"))

# Epsilon progenitors

In [39]:
tp0.pull(14.5, 15.5, data="celltype", subset="Eps. progenitors", key_added="Eps_progenitors_pull")

In [40]:
eps_prog_drivers = tp0.compute_feature_correlation(obs_key="Eps_progenitors_pull", layer="ATAC_imputed", features=genes)

In [41]:
eps_prog_drivers

,Eps_progenitors_pull_corr,Eps_progenitors_pull_pval,Eps_progenitors_pull_qval,Eps_progenitors_pull_ci_low,Eps_progenitors_pull_ci_high
Nefm,0.250270,3.880069e-135,5.689345e-131,0.231188,0.269159
Gm38655,0.242419,1.416437e-126,1.038461e-122,0.223261,0.261389
Fam107b,0.231917,1.289612e-115,6.303194e-112,0.212662,0.250991
Cdkn1a,0.218448,2.262480e-102,8.293687e-99,0.199075,0.237650
Rgs17,0.182547,2.291607e-71,6.720366e-68,0.162895,0.202053
...,...,...,...,...,...
Hmgn3,-0.119755,2.488138e-31,6.633377e-29,-0.139670,-0.099744
Abcc8,-0.123951,1.822160e-33,5.343667e-31,-0.143843,-0.103958
Pax6,-0.124209,1.339367e-33,4.091486e-31,-0.144100,-0.104217
Chgb,-0.137159,1.103026e-40,5.776312e-38,-0.156976,-0.117231


In [42]:
eps_prog_drivers.to_csv(os.path.join(output_dir, "driver_eps_prog_genes.csv"))

In [43]:
eps_prog_drivers_tf = tp0.compute_feature_correlation(obs_key="Eps_progenitors_pull", layer="ATAC_imputed", features="mouse")

In [44]:
eps_prog_drivers_tf

,Eps_progenitors_pull_corr,Eps_progenitors_pull_pval,Eps_progenitors_pull_qval,Eps_progenitors_pull_ci_low,Eps_progenitors_pull_ci_high
Prox1,0.136222,3.796674e-40,5.041983e-37,0.116289,0.156045
Neurod2,0.133732,9.721976e-39,6.455392e-36,0.113787,0.153570
Nkx2-2,0.133163,2.022848e-38,8.954475e-36,0.113215,0.153004
Gadd45a,0.109949,1.241690e-26,2.748274e-24,0.089895,0.129913
Setbp1,0.095177,2.533480e-20,3.738291e-18,0.075068,0.115209
...,...,...,...,...,...
Fev,-0.092647,2.469954e-19,3.280099e-17,-0.112690,-0.072530
Tagln2,-0.095792,1.443500e-20,2.396209e-18,-0.115821,-0.075685
Mlxipl,-0.097277,3.653239e-21,6.930716e-19,-0.117299,-0.077175
Hmgn3,-0.119755,2.488138e-31,6.608495e-29,-0.139670,-0.099744


In [45]:
eps_prog_drivers_tf.to_csv(os.path.join(output_dir, "driver_eps_prog_tfs.csv"))

In [46]:
eps_prog_drivers_peaks = tp0.compute_feature_correlation(obs_key="Eps_progenitors_pull", layer="ATAC_imputed", features=peaks)

In [47]:
eps_prog_drivers_peaks

,Eps_progenitors_pull_corr,Eps_progenitors_pull_pval,Eps_progenitors_pull_qval,Eps_progenitors_pull_ci_low,Eps_progenitors_pull_ci_high
14-79596355-79597272,0.260005,3.351167e-146,4.216304e-141,0.241021,0.278791
12-27615808-27616636,0.250653,1.454797e-135,9.151838e-131,0.231575,0.269538
5-150823073-150823955,0.249905,9.867274e-135,4.138203e-130,0.230819,0.268798
7-136365546-136366501,0.248653,2.390761e-133,7.519901e-129,0.229555,0.267559
6-16486958-16487859,0.245785,3.303436e-130,8.312501e-126,0.226660,0.264721
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [48]:
eps_prog_drivers_peaks.to_csv(os.path.join(output_dir, "driver_eps_prog_peaks.csv"))

# Fev+ Delta

In [49]:
tp0.pull(14.5, 15.5, data="celltype", subset="Fev+ Delta", key_added="Fev_delta_pull")

In [50]:
fev_delta_drivers =  tp0.compute_feature_correlation(obs_key="Fev_delta_pull", layer="ATAC_imputed", features=genes)

In [51]:
fev_delta_drivers

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high
Mboat4,0.228281,5.946502e-112,8.719355e-108,0.208994,0.247391
Isl1,0.203809,5.495934e-89,4.029344e-85,0.184316,0.223142
Nefm,0.180338,1.172858e-69,5.732538e-66,0.160671,0.199862
Cdkn1a,0.176221,1.567543e-66,5.746219e-63,0.156526,0.195776
Rgs17,0.169195,2.236002e-61,6.557299e-58,0.149454,0.188802
...,...,...,...,...,...
Ptprf,-0.075132,3.257575e-13,6.922583e-11,-0.095241,-0.054962
Vim,-0.082998,8.332696e-16,2.262635e-13,-0.103079,-0.062850
Ttr,-0.084361,2.792219e-16,9.098292e-14,-0.104436,-0.064217
Airn,-0.085679,9.529809e-17,3.457488e-14,-0.105750,-0.065540


In [52]:
fev_delta_drivers.to_csv(os.path.join(output_dir, "drivers_fev_delta_genes.csv"))

In [53]:
fev_delta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Fev_delta_pull", layer="ATAC_imputed", features="mouse")

In [54]:
fev_delta_drivers_tfs

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high
Isl1,0.203809,5.495934e-89,7.298601e-86,0.184316,0.223142
Peg3,0.116088,1.587288e-29,1.053959e-26,0.096060,0.136022
Hhex,0.112477,8.356448e-28,3.699121e-25,0.092434,0.132429
Fhl2,0.100312,2.061943e-22,6.845649e-20,0.080221,0.120321
Rfx6,0.080482,5.994171e-15,1.592052e-12,0.060326,0.100571
...,...,...,...,...,...
Cat,-0.049412,1.711000e-06,7.574026e-05,-0.069596,-0.029187
Mafb,-0.054804,1.108187e-07,6.131968e-06,-0.074975,-0.034589
Rora,-0.055935,6.035856e-08,3.643462e-06,-0.076102,-0.035721
Sox5,-0.056174,5.299873e-08,3.351539e-06,-0.076340,-0.035961


In [55]:
fev_delta_drivers_tfs.to_csv(os.path.join(output_dir, "drivers_fev_delta_tfs.csv"))

In [56]:
fev_delta_drivers_peaks = tp0.compute_feature_correlation(obs_key="Fev_delta_pull", layer="ATAC_imputed", features=peaks)

In [57]:
fev_delta_drivers_peaks

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high
13-63992524-63993343,0.209319,6.663499e-94,8.383748e-89,0.189870,0.228604
10-69640253-69641178,0.208109,8.236808e-93,5.181611e-88,0.188651,0.227404
10-64082077-64082994,0.207002,8.110833e-92,3.401575e-87,0.187534,0.226307
8-69030907-69031818,0.202895,3.477758e-88,1.093894e-83,0.183395,0.222236
13-34956731-34957646,0.200222,7.279153e-86,1.831668e-81,0.180702,0.219586
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [58]:
fev_delta_drivers_peaks.to_csv(os.path.join(output_dir, "drivers_fev_delta_peaks.csv"))

# Alpha

In [59]:
tp0.pull(14.5, 15.5, data="refinement", subset="Alpha", key_added="Alpha_pull")

In [60]:
alpha_drivers = tp0.compute_feature_correlation(obs_key="Alpha_pull", layer="ATAC_imputed", features=genes)

In [61]:
alpha_drivers 

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high
Cltrn,0.618957,0.000000e+00,0.000000e+00,0.606304,0.631297
Gcg,0.604927,0.000000e+00,0.000000e+00,0.591926,0.617614
Pou6f2,0.572761,0.000000e+00,0.000000e+00,0.558991,0.586214
Gria2,0.561378,0.000000e+00,0.000000e+00,0.547348,0.575093
Irx2,0.557938,0.000000e+00,0.000000e+00,0.543830,0.571731
...,...,...,...,...,...
Btg2,-0.290914,1.014117e-184,1.906409e-182,-0.309345,-0.272265
Pard3,-0.296613,2.097952e-192,4.101636e-190,-0.314974,-0.278029
Nkx6-1,-0.299304,4.236544e-196,8.874349e-194,-0.317633,-0.280753
Ptma,-0.303642,3.834255e-202,8.518436e-200,-0.321916,-0.285142


In [62]:
alpha_drivers.to_csv(os.path.join(output_dir, "drivers_alpha_genes.csv"))

In [63]:
alpha_drivers_tfs = tp0.compute_feature_correlation(obs_key="Alpha_pull", layer="ATAC_imputed", features="mouse")

In [64]:
alpha_drivers_tfs

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high
Pou6f2,0.572761,0.000000e+00,0.000000e+00,0.558991,0.586214
Irx2,0.557938,0.000000e+00,0.000000e+00,0.543830,0.571731
Irx1,0.537579,0.000000e+00,0.000000e+00,0.523020,0.551824
Mctp2,0.507169,0.000000e+00,0.000000e+00,0.491969,0.522060
Smarca1,0.445295,0.000000e+00,0.000000e+00,0.428910,0.461388
...,...,...,...,...,...
Gadd45a,-0.213767,5.599631e-98,3.718155e-96,-0.233012,-0.194355
Hes6,-0.216801,8.179294e-101,5.716896e-99,-0.236018,-0.197414
Neurog3,-0.266852,2.767458e-154,2.625132e-152,-0.285563,-0.247939
Nkx6-1,-0.299304,4.236544e-196,5.114664e-194,-0.317633,-0.280753


In [65]:
alpha_drivers_tfs.to_csv(os.path.join(output_dir, "drivers_alpha_tfs.csv"))

In [66]:
alpha_drivers_peaks = tp0.compute_feature_correlation(obs_key="Alpha_pull", layer="ATAC_imputed", features=peaks)

In [67]:
alpha_drivers_peaks

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high
19-60167711-60168625,0.677911,0.0,0.0,0.666812,0.688708
14-93825518-93826591,0.666811,0.0,0.0,0.655409,0.677910
3-154084552-154085465,0.666098,0.0,0.0,0.654677,0.677216
16-70767846-70768746,0.663499,0.0,0.0,0.652007,0.674686
3-17473827-17474733,0.656660,0.0,0.0,0.644984,0.668029
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [68]:
alpha_drivers_peaks.to_csv(os.path.join(output_dir, "drivers_alpha_peaks.csv"))

# Beta

In [69]:
tp0.pull(14.5, 15.5, data="refinement", subset="Beta", key_added="Beta_pull")

In [70]:
beta_drivers = tp0.compute_feature_correlation(obs_key="Beta_pull", layer="ATAC_imputed", features=genes)

In [71]:
beta_drivers

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high
Papss2,0.671133,0.0,0.0,0.659848,0.682115
Phactr1,0.664747,0.0,0.0,0.653289,0.675901
Sntg1,0.645568,0.0,0.0,0.633599,0.657228
Ppp1r1a,0.633860,0.0,0.0,0.621586,0.645823
Sytl4,0.619853,0.0,0.0,0.607222,0.632170
...,...,...,...,...,...
Rps8,-0.405746,0.0,0.0,-0.422527,-0.388687
Rps9,-0.414618,0.0,0.0,-0.431250,-0.397704
Rpl9,-0.424648,0.0,0.0,-0.441108,-0.407903
Rps3,-0.425683,0.0,0.0,-0.442125,-0.408955


In [72]:
beta_drivers.to_csv(os.path.join(output_dir, "drivers_beta_genes.csv"))

In [73]:
beta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Beta_pull", layer="ATAC_imputed", features="mouse")

In [74]:
beta_drivers_tfs

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high
Mafb,0.595954,0.000000e+00,0.000000e+00,0.582734,0.608859
Pdx1,0.515776,0.000000e+00,0.000000e+00,0.500754,0.530488
Rora,0.500895,0.000000e+00,0.000000e+00,0.485568,0.515915
Mlxipl,0.448731,0.000000e+00,0.000000e+00,0.432407,0.464761
Mafa,0.409968,0.000000e+00,0.000000e+00,0.392978,0.426679
...,...,...,...,...,...
Cbfa2t2,-0.297207,3.235538e-193,2.261470e-191,-0.315561,-0.278630
Setbp1,-0.335121,2.222050e-249,1.967255e-247,-0.352979,-0.317019
Sox4,-0.359916,5.294679e-291,5.859445e-289,-0.377418,-0.342157
Rps4x,-0.367758,5.026251e-305,6.068056e-303,-0.385143,-0.350112


In [75]:
beta_drivers.to_csv(os.path.join(output_dir, "drivers_beta_tfs.csv"))

In [76]:
beta_drivers_peaks = tp0.compute_feature_correlation(obs_key="Beta_pull", layer="ATAC_imputed", features=peaks)

In [77]:
beta_drivers_peaks

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high
10-20505031-20505953,0.787106,0.0,0.0,0.779276,0.794691
6-6536673-6537567,0.780205,0.0,0.0,0.772153,0.788007
10-76859043-76859925,0.775003,0.0,0.0,0.766785,0.782967
18-84450439-84451317,0.774762,0.0,0.0,0.766536,0.782733
14-121888845-121889756,0.772612,0.0,0.0,0.764319,0.780650
...,...,...,...,...,...
Y-2661677-2662608,NaN,NaN,NaN,NaN,NaN
Y-90725776-90726677,NaN,NaN,NaN,NaN,NaN
Y-90732062-90732805,NaN,NaN,NaN,NaN,NaN
Y-90797268-90798083,NaN,NaN,NaN,NaN,NaN


In [78]:
beta_drivers_peaks.to_csv(os.path.join(output_dir, "drivers_beta_peaks.csv"))

In [79]:
tp0.adata.write(os.path.join(output_dir, "adata_driver.h5ad"))